In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import json
import requests
import string
import re   # regex
import nltk # text handling
from nltk.stem import WordNetLemmatizer
from nltk.stem.isri import ISRIStemmer
from textblob import TextBlob # WordCloud

In [2]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas
import seaborn as sns 
from sklearn import preprocessing
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
import re
import string
import matplotlib.cm as cm
from matplotlib import rcParams
from prettytable import PrettyTable
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avitr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# To see the whole tweet text
pd.set_option('display.max_colwidth', None)

# Data Exploration

In [5]:
# load specific columns from the csv file
#col_list = ["created_at", "id", "in_reply_to_status_id", 
 #           "in_reply_to_user_id", "text", "user_screen_name"]

tweets = pd.read_excel('Sentiment Analysis Training.xlsx')
#
 #                 , usecols = col_list)


In [6]:
len(tweets)

133

# Text Preparation

In [7]:
mentions = re.compile('(@[a-zA-Z0-9_]{1,50})') # mentions regex @...
urls = re.compile('http\S+')         # links regex
diacritics = re.compile('ٰ ّ َ ً ُ ٌ ِ ٍ ْـ')       # Tashkeel regex

emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

arabic_punctuations = '''`÷×؛<>()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ«»'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

stopwords = nltk.corpus.stopwords.words('arabic')

def remove_mentions(text):
    text = re.sub(mentions, '', text)
    return text

# removing URLs
def remove_urls(text):
    # twitter converts all links to thier own domain t.co
    text = re.sub(urls, '', text)
    return text

# removing tashkeel
def remove_diacritics(text):
    text = re.sub(r'[\u064b-\u065f]', '', text)
    return text

def remove_emojis(text):
    # https://www.linkedin.com/pulse/extracting-twitter-data-pre-processing-sentiment-using-jayasekara
    return re.sub(emoji_pattern, '', text)

def remove_repeating_char(text):
    # from https://github.com/motazsaad/process-arabic-text/blob/master/clean_arabic_text.py
    return re.sub(r'(.)\1+', r'\1', text)

def lemmatize_text(text):
    # lemmatizes text
    lemmatized_text = []

    # set up arabic lemmatizer Farasa
    url = 'https://farasa.qcri.org/webapi/lemmatization/'
    api_key = "lErIOPgmHZtflLMgIf"

    # set up english lemmatizer
    eng_lemmatizer = nltk.stem.WordNetLemmatizer()

    for word in text:
        # Detect language to use proper lemmatizer
        if TextBlob(word).detect_language() == 'en':
            lemmatized_text.append(eng_lemmatizer.lemmatize(word))
        else:
            payload = {'text': word, 'api_key': api_key}
            data = requests.post(url, data=payload)
            lemmatized_text.append(json.loads(data.text))
    return lemmatized_text

def text_stemming(text):
    return ISRIStemmer().suf32(text)

def remove_punctuations(text):
    # from https://github.com/motazsaad/process-arabic-text/blob/master/clean_arabic_text.py
    translator = str.maketrans(' ', ' ', punctuations_list)
    return text.translate(translator)

def normalize_arabic(text):
    # from https://github.com/motazsaad/process-arabic-text/blob/master/clean_arabic_text.py
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
    
# remove hashtags marks but keep the words itself
def normalize_hashtags(text):
    text = re.sub("#", "", text)
    text = re.sub("_", " ", text)
    text = re.sub("_", " ", text)
    return text

In [8]:
normalized_stopwords = map(normalize_arabic, stopwords)
normalized_stopwords = list(normalized_stopwords)
normalized_stopwords = list(set(normalized_stopwords))
normalized_stopwords[:5]

['انتِ', 'اثنان', 'سبعماءه', 'فاء', 'راي']

In [9]:
# clean text
def text_preprocessing(text):
    text = remove_diacritics(text)
    text = remove_mentions(text)
    text = normalize_hashtags(text)
    text = remove_urls(text)
    text = remove_emojis(text)
    text = remove_repeating_char(text)
    text = remove_punctuations(text)
    text = normalize_arabic(text)
    text = ' '.join(word for word in text.split() if word not in normalized_stopwords)
    return text#.split()

In [10]:
tweets['text']=tweets['text'].apply(str)

In [11]:
tweets['clean_tweet'] = tweets['text'].apply(text_preprocessing)

# Vectorization

In [12]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

word_vectorizer = word_vectorizer.fit(tweets['clean_tweet'])
unigramdataGet = word_vectorizer.transform(tweets['clean_tweet'])
unigramdataGet = unigramdataGet.toarray()

In [13]:
vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1

unigramdata_features.head()

,10,13,5g,90,iphone,my,mystc,stc,stcpay,stcوش,...,١٨,٢٠,٢٠٣٠,٢٤,٢٩,٣خطوط,٣٠١٢,٤ساعات,٥٠,٨سنوات
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
pro = preprocessing.LabelEncoder()
tweets['label_encoded'] = pro.fit_transform(tweets['label'])

In [15]:
for indx,clas in enumerate(pro.classes_):
    print(indx,clas)

0 Negative
1 Neutral
2 Positive


In [16]:
y = tweets['label_encoded']
X = unigramdata_features

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [18]:
X_train.head()

,10,13,5g,90,iphone,my,mystc,stc,stcpay,stcوش,...,١٨,٢٠,٢٠٣٠,٢٤,٢٩,٣خطوط,٣٠١٢,٤ساعات,٥٠,٨سنوات
53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
nb=GaussianNB()
nb= nb.fit(X_train , y_train)
nb

GaussianNB()

In [20]:
y_pred = nb.predict(X_test)
# nb_1=nb.score(X_test, y_test)
print('Accuracy= {:.3f}'.format(nb.score(X_test, y_test)))
print('Accuracy= {:.3f}'.format(nb.score(X_train , y_train)))

Accuracy= 0.593
Accuracy= 0.953


In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73        16
           1       0.50      0.38      0.43         8
           2       0.25      0.33      0.29         3

    accuracy                           0.59        27
   macro avg       0.49      0.49      0.48        27
weighted avg       0.59      0.59      0.59        27



In [22]:
y_pred

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 1, 2, 2, 0, 1,
       0, 0, 0, 1, 0])

# Save the Models

In [23]:
import pickle

pickle.dump(nb, open('classification.pickle', 'wb'))

pickle.dump(word_vectorizer, open('word_vectorizer.pickle', 'wb'))

# read test file for testing

In [24]:
replies = pd.read_excel('test_set.xlsx') # it is the same file with different name

In [25]:
replies['text']=replies['text'].apply(str)

In [26]:
replies['clean_tweet'] = replies['text'].apply(text_preprocessing)

# Load saved model

In [27]:
vectorizer = pickle.load(open('word_vectorizer.pickle','rb'))
model = pickle.load(open('classification.pickle','rb'))

# Use the loaded model

In [28]:
unigramdataGet = vectorizer.transform(replies['clean_tweet'])
unigramdataGet = unigramdataGet.toarray()
unigramdataGet[unigramdataGet>0] = 1

In [29]:
y_pred = model.predict(unigramdataGet)

replies['Predicted_label'] = y_pred
replies['Predicted_label'] = replies['Predicted_label'].replace([0,1,2],['Negative', 'Neutral', 'Positive'])

In [30]:
replies[['text','label','Predicted_label']]

,text,label,Predicted_label
0,@NASEER_OB @ ابشرك بتدخل اليوم العاشر عشان يردوا عليك او يخدمونك ياخي تحس stc مايدرون عن رؤية ٢٠٣٠ 😅 الى الحين ونفس الاخطاء من عشرات السنين,Negative,Negative
1,@NASEER_OB @ ابشرك ٣خطوط نقلتها من @ الى @Mobily وتوبة وماعاد لي رجعة لهم,Positive,Positive
2,@ اخذته على رقمي الثاني من فرع الظهران مول الموضوع ما اخذ ولا عشر دقايق شكراً #stc @stc https://t.co/eAtmgQJKgv,Positive,Negative
3,@s9sb @ من يعوضني ي stc 🥺,Negative,Negative
4,@NASEER_OB @ ترا ماينفع معهم الا انك تروح لاقرب فرع وتتفاهم معهم وقلهم بحول عنكم شوف كيف يرجعو,Neutral,Neutral
...,...,...,...
128,@ @mjeid_0 ياخي ليه ماتردون وتخلصون مشكلتي 🤌🏼,Negative,Negative
129,@ خلاص فعلته بطريقة ثانيه من mystc وحطيته بالرمز اليدوي,Neutral,Neutral
130,@ \nصباح الخير \nألغيت رقمي المفوتر ونزلت لي مديوانية \nسلاماااات .. \nكيف كذا ؟؟,Negative,Negative
131,@ سوا بوست بلس,Neutral,Neutral
